# Tools and Routing

In this notebook, I experiment with the Langchain lesson on Tools and Routing and explore the Marvel API.

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
marvel_private_key = os.environ['MARVEL_PRIVATE_KEY']
marvel_public_key = os.environ['MARVEL_PUBLIC_KEY']

In [7]:
from langchain.agents import tool
from langchain.chat_models import ChatOpenAI
import json
import openai

In [25]:
import requests # used for http requests
import hashlib
import time
from pydantic import BaseModel, Field

timestamp = str(int(time.time()))
hash = hashlib.md5((timestamp + marvel_private_key + marvel_public_key).encode()).hexdigest()

# Define the input schema
class SearchMarvel(BaseModel):
    name: str = Field(..., description="Name of the Marvel character")
    

@tool(args_schema=SearchMarvel)
def get_character_description(name: str) -> json:
    """Fetch the description of the given character."""
    
    BASE_URL = f"http://gateway.marvel.com/v1/public/characters?ts={timestamp}&apikey={marvel_public_key}&hash={hash}"

    # Parameters for the request
    params = {
        'name': name,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        
        # Extract the character description
        character_info = results['data']['results'][0]  # Assuming the character is the first in the results list
        description = character_info.get('description', 'No description available')

        return description
    
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
    
   # return f'Here is the character description: {results}'

In [26]:
get_character_description.name

'get_character_description'

In [27]:
get_character_description.description

"get_character_description(name: str) -> <module 'json' from '/usr/local/Cellar/python@3.9/3.9.17_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/json/__init__.py'> - Fetch the description of the given character."

In [28]:
get_character_description.args

{'name': {'title': 'Name',
  'description': 'Name of the Marvel character',
  'type': 'string'}}

In [29]:
from langchain.tools.render import format_tool_to_openai_function

In [30]:
#convert the tool to an openai function
format_tool_to_openai_function(get_character_description)

{'name': 'get_character_description',
 'description': "get_character_description(name: str) -> <module 'json' from '/usr/local/Cellar/python@3.9/3.9.17_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/json/__init__.py'> - Fetch the description of the given character.",
 'parameters': {'title': 'SearchMarvel',
  'type': 'object',
  'properties': {'name': {'title': 'Name',
    'description': 'Name of the Marvel character',
    'type': 'string'}},
  'required': ['name']}}

In [31]:
get_character_description({"name": "Captain America"})

"Vowing to serve his country any way he could, young Steve Rogers took the super soldier serum to become America's one-man army. Fighting for the red, white and blue for over 60 years, Captain America is the living, breathing symbol of freedom and liberty."

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [32]:
# my code
#Lets see if it can distinguish between functions
functions = [
    format_tool_to_openai_function(f) for f in [
        get_character_description, #add more tools here
    ]
]
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo").bind(functions=functions)

In [33]:
model.invoke("Who is Captain America?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_character_description', 'arguments': '{\n  "name": "Captain America"\n}'}})

In [34]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [35]:
chain.invoke({"input": "who is Iron Man?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_character_description', 'arguments': '{\n  "name": "Iron Man"\n}'}})

In [36]:
#Add a parser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [37]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [38]:
result = chain.invoke({"input": "who is Iron Man?"})

In [39]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [40]:
result.tool

'get_character_description'

In [41]:
result.tool_input

{'name': 'Iron Man'}

In [42]:
# Now run the tool
get_character_description(result.tool_input)

'Wounded, captured and forced to build a weapon by his enemies, billionaire industrialist Tony Stark instead created an advanced suit of armor to save his life and escape captivity. Now with a new outlook on life, Tony uses his money and intelligence to make the world a safer, better place as Iron Man.'

Using third party api's with a language model allows a user to interact with the api in natural language. The LLM can then infer, enhance, combine the api outputs.

In [ ]:
result = chain.invoke({"input": "hi!"})

In [ ]:
type(result)

In [ ]:
result.return_values

In [43]:
#differentiate betwween when a tool is called or not called
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish): # if no tool is called
        return result.return_values['output']
    else: # if a tool is called
        tools = {
            "get_character_description": get_character_description,
        }
        return tools[result.tool].run(result.tool_input)

When you add a route, you get the actual "answer"

In [60]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [45]:
result = chain.invoke({"input": "Who is Captain America?"})

In [46]:
result

"Vowing to serve his country any way he could, young Steve Rogers took the super soldier serum to become America's one-man army. Fighting for the red, white and blue for over 60 years, Captain America is the living, breathing symbol of freedom and liberty."

In [ ]:
chain.invoke({"input": "hi!"})

Get all characters and their descriptions, with pagination

In [51]:

import requests

# Initialize an empty dictionary to store character data
character_dict = {}

# Assuming these variables are defined: timestamp, marvel_public_key, hash
base_url = "http://gateway.marvel.com/v1/public/characters"

offset = 0
limit = 20  # Adjust if needed, based on API's allowed limits

while True:
    response = requests.get(f"{base_url}?ts={timestamp}&apikey={marvel_public_key}&hash={hash}&offset={offset}&limit={limit}")
    data = response.json()

    characters = data['data']['results']
    for character in characters:
        name = character['name']
        description = character.get('description', 'No description available')
        
        # Store the name and description in the dictionary
        character_dict[name] = description

    total = data['data']['total']
    count = data['data']['count']

    if offset + count >= total:
        break  # Exit the loop if all pages have been processed

    offset += count

# character_dict now contains the names and descriptions


In [ ]:
character_dict

In [ ]:
type(character_dict)

In [54]:
#Print the last 100 entries
# Convert the dictionary items to a list and slice the last 100 items
last_100_characters = list(character_dict.items())[-100:]

# Print each name and description pair
for name, description in last_100_characters:
    print(f"Name: {name}\nDescription: {description}\n")


Name: Violations
Description: 

Name: Viper
Description: After the apparent death of Baron von Strucker, Viper took the name Madame Hydra and took control of the New York-based faction of Hydra.

Name: Virginia Dare
Description: 

Name: Vision
Description: The metal monstrosity called Ultron created the synthetic humanoid known as the Vision from the remains of the original android Human Torch of the 1940s to serve as a vehicle of vengeance against the Avengers.

Name: Vivisector
Description: 

Name: Vulcan (Gabriel Summers)
Description: Vulcan is the third Summers brother and an Omega-level mutant who was thought long-dead by Professor X.

Name: Vulture (Adrian Toomes)
Description: Adrian Toomes is a former electronics engineer who employs a special harness of his own design that allows him to fly and endows him with enhanced strength.

Name: Vulture (Blackie Drago)
Description: Adrian Toomes, the Vulture, told his cellmate, Blackie Drago, the location of his Vulture costume.

Name: W

In [59]:
# Save as JSON
with open('character_dict.json', 'w') as file:
    json.dump(character_dict, file)

In [ ]:
# Playing around

In [61]:
chain.invoke({"input": "Who is the wasp?"})

'When Janet Van Dyne\'s father died, she convinced her father\'s associate Hank Pym to give her a supply of "Pym particles"; Pym also subjected her to a procedure which granted her the ability to, upon shrinking, grow wings and fire blasts of energy, which she called her "wasp\'s stings."'

In [9]:
#create get_completion helper function
def get_completion(prompt, model="gpt-4-1106-preview"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature =0, #this is the degree of randomness 
    )
    return response.choices[0].message["content"]

In [10]:
#Load the json file
# Load the JSON file
with open('marvel_character_dict.json', 'r') as file:
    marvel_character_dict = json.load(file)

In [11]:
#Interrogating the Marvel characters
prompt = f"""
The following Marvel character dictionary is delimited with triple backticks.
Which of the characters have a close relation to insects?

Character dictionary: '''{marvel_character_dict}'''
"""

response = get_completion(prompt)
print(response)

Based on the character dictionary provided, the following characters have a close relation to insects:

1. **Spider-Man (Peter Parker)** - Bitten by a radioactive spider, he gained the speed, strength, and powers of a spider.
2. **Wasp** - Janet Van Dyne can shrink to insect size, has wings, and can fire bio-electric energy blasts.
3. **Ant-Man (Hank Pym)** - Has the ability to shrink to the size of an ant and communicate with ants.
4. **Ant-Man (Scott Lang)** - Inherits the Ant-Man suit and powers from Hank Pym.
5. **Ant-Man (Eric O'Grady)** - Another character who takes on the Ant-Man mantle with similar powers.
6. **Yellowjacket (Hank Pym)** - Another superhero identity of Hank Pym, with similar powers to Ant-Man.
7. **Yellowjacket (Rita DeMara)** - A character who stole the Yellowjacket costume and powers.
8. **Spider-Woman (Jessica Drew)** - Has powers similar to Spider-Man, including wall-crawling and a venom blast.
9. **Spider-Girl (May Parker)** - The daughter of Spider-Man, wi